In [1]:
from data.visa_kor_docs import VisaDocuments
lang_docs = VisaDocuments

In [ ]:
# with open("data/E-9_Visa_Guide_한국어.json", "w") as fp:
#     json.dump(arr_docs, fp)

In [10]:
import json
# from langchain.schema import Document
from llama_index.core import Document
from llama_index.core import VectorStoreIndex, Settings

def get_doc(doc, page):
    return Document(
        text=doc.page_content, 
        metadata={
            "page_label": page,
            "document_title": doc.metadata["document_title"],
            "questions_this_excerpt_can_answer": doc.metadata["questions_this_excerpt_can_answer"],
            "section_summary": doc.metadata["section_summary"],
            "excerpt_keywords": doc.metadata["excerpt_keywords"],
        },
    )

arr_docs = [get_doc(doc, idx + 1) for idx, doc in enumerate(lang_docs)]

print(len(arr_docs))
doc_nodes = Settings.node_parser.get_nodes_from_documents(arr_docs)

20


In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb

In [ ]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_files=["data/visa_docs/E-9 Visa Guide_한국어2.pdf"])
docs = reader.load_data()

In [18]:
""" SAVE TO LOCAL"""
db = chromadb.PersistentClient(path="./storage4")
chroma_collection = db.get_or_create_collection("E9_VISA")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents=arr_docs, 
    storage_context=storage_context,
    show_progress=True
)

Parsing nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/45 [00:00<?, ?it/s]

In [19]:
""" Load DB From Local"""

# db = chromadb.PersistentClient(path="./storage")
# chroma_collection = db.get_collection("E-9_VISA")

db = chromadb.PersistentClient(path="./storage4")
chroma_collection = db.get_collection("E9_VISA")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [15]:
from llama_index.core.response.notebook_utils import display_source_node

def print_source_node(retrievals):
    for n in retrievals:
        display_source_node(n, source_length=1500)

base_retriever = index.as_retriever(similarity_top_k=3)

In [24]:
retrievals = base_retriever.retrieve("E-9 비자 신청 시 필요한 서류는 무엇인가요?")
print_source_node(retrievals)

**Node ID:** e820fedd-48fe-49b2-a801-f5a469107fa0<br>**Similarity:** 0.8776880133633795<br>**Text:** ## 사증발급인정서를 받아야만 E-9 사증을 발급 받을수있음
        ● 비전문취업(E-9) 사증은 출입국ᆞ외국인청(사무소ᆞ출장소)장이 발급한 사증발급인정서에 의해서만 사증을 발급할 수 있음<br>

**Node ID:** 0284633d-92ec-46ac-908e-a4f140a7322b<br>**Similarity:** 0.8543593242744546<br>**Text:** 지방자치단체, 교육기관, 법인의 대표자 변경의 경우 신고대상에서 제외)
                ■ 근무처의 명칭 변경
                ■ 근무처의 이전으로 그 소재지가 변경된 때
        ● 제출서류
            ○ 고용·연수 외국인 변동사유 발생신고서(별지 32호 서식), 외국인등록증 사본(소재불명 신고의 경우에는 대상자의 휴대폰 번호, 예상소재지 등을 반드시 포함하여야 함)
            ○ 사업자등록증 사본
            ○ 고용주 신분증(고용주 직접 신고 시) ④ 회사 직원 대리 신청 : 고용주 위임장, 재직증명서, 직원 신분증<br>

**Node ID:** a37f5ea5-f6c0-4903-9bf1-878fb54e1cfe<br>**Similarity:** 0.8529940706209486<br>**Text:** 어업허가증 또는 어업면허증 또는 내수면어업신고필증
                    ● 천일염 생산 및 암염 채취업 : 염제조업허가증사본*
                        ○ 염전임차인의 경우에는 시장.군수가 발급하는 염제조업 확인서와 임대차계약서, 임대인의 염제조허가증
                ■ 서비스업
                    ● 추가서류 없음
            ○ 출입국ᆞ외국인청(사무소ᆞ출장소)장은 초청의 진정성, 초청자 및 피초청자의 자격 확인 등을 심사하기 위해 첨부서류의 일부를 가감할 수 있음
            ○ 한식 음식점업, 호텔‧숙박업, 임업, 광업은 고용허가서 발급 시점 이후 첨부서류 안내 예정<br>

In [ ]:
retrievals = base_retriever.retrieve("외국인등록 시 제출해야 하는 마약검사확인서의 구체적인 기준과 유의사항은 무엇인가요?")
print_source_node(retrievals)

In [22]:
# retrievals = base_retriever.retrieve("고용허가제 법률적 개요와 고용허가제 선정국가에 대해서 간단하게 요약해서 설명해줘")
retrievals = base_retriever.retrieve("E-9 비자의 노동자가 취업할수 있는 업종은 어떤게 있어?")

In [23]:
for re in retrievals:
    print(f"{re.metadata['page_label']}")
    # print(f"{re.metadata['document_title']}")
    print(f"{re.score}\n")
    print(f"{re.text}\n\n\n")

4
0.8987540197320741

잡지 및 기타 인쇄물 출판업
                    ● 음악 및 기타 오디오 출판업
                ■ 아래 업종의 표준직업분류상 하역 및 적재 단순종사자
                    ● 폐기물 수집, 운반, 처리 및 원료 재생업
                        ○ ※ 단, ‘폐기물 수집, 운반, 처리 및 원료 재생업’의 경우는 폐기물 분류 업무도 포함
                    ● 음식료품 및 담배 중개업
                    ● 기타 신선 식품 및 단순 가공식품 도매업
                    ● 택배업
                    ● 기타 항공 운송지원 서비스업(52939) 중 ｢항공사업법｣ 시행규칙 제5조제2호에 따른 항공기하역업체
                    ● 항공 및 육상화물취급업 중 ｢축산물 위생관리법｣ 제2조에 따른 식육을 운반하는 업체, ｢생활물류서비스 산업발전법｣ 제2조제3호가목에 따른 택배서비스업체
        
        ● 신규 추가된 ‘한식 음식점업*’, ‘호텔‧숙박업’, ‘임업’, ‘광업’은 ’24년도 고용허가서 발급 시점 이후부터 사증발급인정서 발급 신청 가능



4
0.8984714298092024

■ 축산업
                ■ 농축산서비스업: 작물재배 및 축산관련 서비스업
        ● 어업
            ○ 체류자격: E-9-4
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조건)
                ■ 연근해어업: 연안어업ᆞ근해어업
                    ● 선원법 적용을 받지 않는 20톤 미만 어선 종사자에 한함
                ■ 양식어업
                ■ 소금채취업
        ● 임업
            ○ 체류자격:
            ○ 적용범위(E-9 근로자를 고용할 수 있는 조